In [19]:
from __future__ import print_function
%matplotlib inline
import fluiddyn as fld

# Tutorial: running a simulation (user perspective)

In this tutorial, I'm going to show how to run a simple simulation with a solver already written that solves the 2 dimensional Navier-Stokes equations. I'm also going to present some useful concepts and objects used in the simulation part of FluidDyn.

## Importing a solver

In [20]:
from fluidsim.solvers.ns2d import solver
solver.__file__

'/home/pierre/Dev/fluidsim/fluidsim/solvers/ns2d/solver.pyc'

There is also another quite convenient way to import a solver from a string, here "NS2D":

In [21]:
solver = fld.simul.import_module_solver_from_key('NS2D')
solver.__file__

'/home/pierre/Dev/fluidsim/fluidsim/solvers/ns2d/solver.pyc'

## Create an instance of the class Parameters

The next step is to create an object `params` from the information contained in the module `solver`:

In [22]:
params = solver.Simul.create_default_params()

In [23]:
[attr for attr in dir(params) if not attr.startswith('_')]

['FORCING',
 'NEW_DIR_RESULTS',
 'ONLY_COARSE_OPER',
 'beta',
 'forcing',
 'init_fields',
 'nu_2',
 'nu_4',
 'nu_8',
 'nu_m4',
 'oper',
 'output',
 'short_name_type_run',
 'time_stepping']

In [24]:
[attr for attr in dir(params) if attr.startswith('_') and not attr.startswith('__')]

['_attribs',
 '_load_from_elemxml',
 '_load_from_hdf5_file',
 '_load_from_hdf5_objet',
 '_load_from_xml_file',
 '_make_dict',
 '_make_element_xml',
 '_make_xml_text',
 '_print_as_xml',
 '_save_as_hdf5',
 '_save_as_xml',
 '_set_as_child',
 '_set_attrib',
 '_set_attribs',
 '_set_child',
 '_set_internal_attr',
 '_tag',
 '_tag_children']

In [25]:
print(type(params.nu_2))
print(type(params.output))

<type 'float'>
<class 'fluidsim.base.params.Parameters'>


In [26]:
[attr for attr in dir(params.output) if not attr.startswith('_')]

['HAS_TO_SAVE',
 'ONLINE_PLOT_OK',
 'increments',
 'period_refresh_plots',
 'periods_plot',
 'periods_print',
 'periods_save',
 'phys_fields',
 'spatial_means',
 'spect_energy_budg',
 'spectra']

Therefore, the object `params` contains a tree of parameters. This tree can be represented as xml code:

In [27]:
print(params)


<params ONLY_COARSE_OPER="False" short_name_type_run="" beta="0.0" nu_2="0.0"
        NEW_DIR_RESULTS="True" nu_4="0.0" nu_8="0.0" FORCING="False"
        nu_m4="0.0">
  <oper type_fft="FFTWCY" nx="48" ny="48" coef_dealiasing="0.6666666666666666"
        TRANSPOSED_OK="True" Lx="8" Ly="8"/>  

  <init_fields available_types="['from_file', 'noise', 'constant', 'jet',
               'manual', 'dipole', 'from_simul']" type="constant">
    <from_file path=""/>  

    <noise length="0" velo_max="1.0"/>  

    <constant value="1.0"/>  

  </init_fields>

  <forcing nkmax_forcing="5" nkmin_forcing="4" key_forced="rot_fft"
           available_types="['proportional', 'random']" type=""
           forcing_rate="1">
    <random type_normalize="2nd_degree_eq"/>  

  </forcing>

  <time_stepping type_time_scheme="RK4" it_end="10" USE_CFL="True" deltat0="0.2"
                 t_end="10.0" USE_T_END="True"/>  

  <output period_refresh_plots="1" ONLINE_PLOT_OK="True" HAS_TO_SAVE="True">
    <period

## Set the parameters for your simulation

The user can change any parameters

In [28]:
params.nu_2 = 1.
params.FORCING = False
params.init_fields.type

'constant'

In [29]:
try:
    params.this_param_does_not_exit = 10
except AttributeError as e:
    print('AttributeError:', e)

AttributeError: this_param_does_not_exit is not already set in params.
The attributes are: set(['ONLY_COARSE_OPER', 'short_name_type_run', 'beta', 'nu_2', 'NEW_DIR_RESULTS', 'nu_4', 'nu_8', 'FORCING', 'nu_m4'])
To set a new attribute, use _set_attrib or _set_attribs.


This behaviour is much safer than using a text file or a python file for the parameters.   

## Instantiate a simulation object

The next step is to create a simulation object (instance of the class solver.Simul) with the parameters in `params`:

In [30]:
sim = solver.Simul(params)

*************************************
Program FluidDyn

solver NS2D, RK4 and sequenciel,
type fft: FFTWCY
nx =     48 ; ny =     48
Lx = 8. ; Ly = 8.
path_run =
/home/pierre/Sim_data/NS2D_L=8.x8._48x48_2015-04-19_18-17-50
init_fields.type: constant
Initialization outputs:
<class 'fluidsim.base.output.increments.Increments'> increments
<class 'fluidsim.base.output.phys_fields.PhysFieldsBase'> phys_fields
<class 'fluidsim.solvers.ns2d.output.spectra.SpectraNS2D'> spectra
<class 'fluidsim.solvers.ns2d.output.spatial_means.SpatialMeansNS2D'> spatial_means
<class 'fluidsim.solvers.ns2d.output.spect_energy_budget.SpectralEnergyBudgetNS2D'> spect_energy_budg

Memory usage at the end of init. (equiv. seq.): 75.234375 Mo
Size of state_fft (equiv. seq.): 0.0192 Mo


which initializes everything needed to run the simulation. The object `sim` has few attributes:

In [31]:
[attr for attr in dir(sim) if not attr.startswith('_')]

['InfoSolver',
 'compute_freq_diss',
 'create_default_params',
 'info',
 'info_solver',
 'init_fields',
 'name_run',
 'oper',
 'output',
 'params',
 'state',
 'tendencies_nonlin',
 'time_stepping']

In [32]:
print(sim.info.__class__)
print([attr for attr in dir(sim.info) if not attr.startswith('_')])

<class 'fluiddyn.util.paramcontainer.ParamContainer'>
['params', 'solver']


In [33]:
print(sim.info.solver)


<solver class_name="Simul" module_name="fluidsim.solvers.ns2d.solver"
        short_name="NS2D">
  <classes>
    <Operators class_name="OperatorsPseudoSpectral2D"
               module_name="fluidsim.operators.operators"/>  

    <InitFields class_name="InitFieldsNS2D"
                module_name="fluidsim.solvers.ns2d.init_fields">
      <classes>
        <from_file class_name="InitFieldsFromFile"
                   module_name="fluidsim.base.init_fields"/>  

        <noise class_name="InitFieldsNoise"
               module_name="fluidsim.solvers.ns2d.init_fields"/>  

        <constant class_name="InitFieldsConstant"
                  module_name="fluidsim.base.init_fields"/>  

        <jet class_name="InitFieldsJet"
             module_name="fluidsim.solvers.ns2d.init_fields"/>  

        <manual class_name="InitFieldsManual"
                module_name="fluidsim.base.init_fields"/>  

        <dipole class_name="InitFieldsDipole"
                module_name="fluidsim.solvers.ns2

## Run the simulation

We are ready to launch the simulation!

In [34]:
sim.time_stepping.start()

*************************************
Beginning of the computation
save state_phys in file state_phys_t=000.000_it=0.hd5
    compute until t =         10
it =      0 ; t =          0 ; deltat  =   0.083333
              energy = 0.000e+00 ; Delta energy = +0.000e+00

it =      6 ; t =    1.08333 ; deltat  =        0.2
              energy = 0.000e+00 ; Delta energy = +0.000e+00
              estimated remaining duration =     0.332 s
it =     12 ; t =    2.28333 ; deltat  =        0.2
              energy = 0.000e+00 ; Delta energy = +0.000e+00
              estimated remaining duration =     0.272 s
it =     17 ; t =    3.28333 ; deltat  =        0.2
              energy = 0.000e+00 ; Delta energy = +0.000e+00
              estimated remaining duration =     0.362 s
it =     22 ; t =    4.28333 ; deltat  =        0.2
              energy = 0.000e+00 ; Delta energy = +0.000e+00
              estimated remaining duration =     0.268 s
it =     27 ; t =    5.28333 ; deltat  =        0.2
